# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Dataset

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
print(ws)

# dataset key info
key = 'nba_3pt_data'
data_desc = 'NBA game data from 2015 to 2023 containing 3 point percentages.'

# checking if dataset is already registered
if key in ws.datasets.keys():
    nba_dataset = ws.datasets[key]
else:
    # get data from url
    nba_data_url = 'https://raw.githubusercontent.com/mattwatson50/udacity_azure_ml_final/main/data/nba_3pt_data.csv'
    dataset = Dataset.Tabular.from_delimited_files(nba_data_url)        
    # register dataset
    dataset = dataset.register(workspace=ws,
                               name=key,
                               description=data_desc)

# getting nba dataset
nba_df = nba_dataset.to_pandas_dataframe()
print(nba_df.head())

experiment_name = 'hd_nba_3pt'
experiment=Experiment(ws, experiment_name)

Workspace.create(name='quick-starts-ws-257351', subscription_id='d4ad7261-832d-46b2-b093-22156001df5b', resource_group='aml-quickstarts-257351')
   season_id matchup_home     season_type  min         team_name_home  \
0      22015  ATL vs. DET  Regular Season  240          Atlanta Hawks   
1      22015  GSW vs. NOP  Regular Season  240  Golden State Warriors   
2      22015  CHI vs. CLE  Regular Season  240          Chicago Bulls   
3      22015  BOS vs. PHI  Regular Season  240         Boston Celtics   
4      22015  MIA vs. CHA  Regular Season  240             Miami Heat   

   fg3m_home  fg3a_home  fg3_pct_home wl_home        team_name_away  \
0          8         27         0.296       L       Detroit Pistons   
1          9         30         0.300       W  New Orleans Pelicans   
2          7         19         0.368       W   Cleveland Cavaliers   
3          8         24         0.333       W    Philadelphia 76ers   
4         12         20         0.600       W     Charlotte H

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [3]:
# creating cpu cluster unless it's already created
cluster_name = "proj-compute-cluster"

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Cluster already created.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS3_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Cluster already created.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [32]:
# setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='./conda_dependencies.yml')

# early termination policy
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

# setting param sampling
param_sampling = RandomParameterSampling(
    {
        '--C': choice(0.01, 0.1, 0.5, 1, 2, 5),
        '--max_iter': choice(20, 40, 80, 120, 160, 200, 250)
    }
)

# creating hyperdrive config
estimator = ScriptRunConfig(source_directory="./",
                            script='./train.py',
                            compute_target=cpu_cluster,
                            environment=sklearn_env)

hyperdrive_run_config = HyperDriveConfig(run_config=estimator,
                                         hyperparameter_sampling=param_sampling,
                                         policy=early_termination_policy,
                                         primary_metric_name='Accuracy',
                                         primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                         max_total_runs=40,
                                         max_concurrent_runs=4)

In [33]:
# run hyperdrive experiment
hyperdrive_run = experiment.submit(config=hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [34]:
# get run details
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [35]:
# get your best run and save the model from that run
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()
print('Best Run Id: ', best_run.id)
print('\n')
print('Best Run Metrics:', best_run_metrics)

Best Run Id:  HD_f8dfa404-d6ad-4713-8b75-93712d8a613f_39


Best Run Metrics: {'Regularization Strength:': 0.01, 'Max iterations:': 20, 'Accuracy': 0.7063211621515508}


In [47]:
# saving best model
best_run.get_file_names()
best_run.download_file('outputs/model.joblib', output_file_path='./')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [48]:
# register the model
model_name = 'nba_3pt_hyperdrive_lr'
model_path = 'outputs/model.joblib'
model_description = 'Logistic Regression Model for NBA 3PT Percentage Win Analysis'
registered_model = best_run.register_model(model_name=model_name,
                                           model_path=model_path,
                                           description=model_description)

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

